In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# If you are running on Google Colab, uncomment below to install the necessary dependencies 
# before beginning the exercise.

print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.8.0.dev5-cp36-cp36m-manylinux1_x86_64.whl
!pip install -q ray[debug]

# A hack to force the runtime to restart, needed to include the above dependencies.
print("Done installing! Restarting via forced crash (this is not an issue).")
import os
os._exit(0)

In [ ]:
# If you are running on Google Colab, please install TensorFlow 2.0 by uncommenting below..

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
!pip3 install ray xlrd pandas  tensorboardX

In [ ]:
print("Setting up colab environment")
!pip3 uninstall -y -q pyarrow
!pip3 install -q ray[debug]


In [ ]:
# !pip3 install tensorflow keras scikit-learn pandas

In [ ]:
import numpy as np
np.random.seed(0)

import tensorflow as tf
try:
    tf.get_logger().setLevel('INFO')
except Exception as exc:
    print(exc)
import warnings
warnings.simplefilter("ignore")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

import ray
from ray import tune
from ray.tune.examples.utils import get_iris_data

import inspect
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
!pip3 install ray[tune] ray[rllib] matplotlib

In [ ]:
import os
import pandas as pd
path = "/content/drive/My Drive/excels/"
files = os.listdir(path)

files_xls = [f for f in files if f[-3:] == 'xls']
df = pd.DataFrame()
for f in files_xls:
    fp=path+f
    data = pd.read_excel(fp, 'Sheet')
    data.columns = ['Time', 'output','outputPitch','outputVolm','Player1BPM','Player1Pitch','Player1Volm', 'Player2BPM','Player2Pitch','Player2Volm', 'Player3BPM','Player3Pitch','Player3Volm', 'Player4BPM','Player4Pitch','Player4Volm']
    df = df.append(data)   
df.columns = ['Time', 'output','outputPitch','outputVolm','Player1BPM','Player1Pitch','Player1Volm', 'Player2BPM','Player2Pitch','Player2Volm', 'Player3BPM','Player3Pitch','Player3Volm', 'Player4BPM','Player4Pitch','Player4Volm']
y =  df[['output']]
X = df[['Time','Player1BPM','Player1Pitch','Player1Volm', 'Player2BPM','Player2Pitch','Player2Volm', 'Player3BPM','Player3Pitch','Player3Volm', 'Player4BPM','Player4Pitch','Player4Volm']]

In [ ]:
# df = pd.read_excel('/content/drive/My Drive/excels/A Classic Education - NightOwl.stem.xls', 'Sheet')
# df.columns = ['Time', 'output','outputPitch','outputVolm','Player1BPM','Player1Pitch','Player1Volm', 'Player2BPM','Player2Pitch','Player2Volm', 'Player3BPM','Player3Pitch','Player3Volm', 'Player4BPM','Player4Pitch','Player4Volm']
# y =  df[['output']]
# X = df[['Time','Player1BPM','Player1Pitch','Player1Volm', 'Player2BPM','Player2Pitch','Player2Volm', 'Player3BPM','Player3Pitch','Player3Volm', 'Player4BPM','Player4Pitch','Player4Volm']]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1,shuffle=False,random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(train_x)
X_test_scaled = scaler_X.transform(test_x)

In [ ]:
train_x = np.array(X_train_scaled) #X_train_scaled
train_x = train_x[:, :, None]
print(train_x.shape)
test_x = np.array(X_test_scaled)#X_test_scaled
test_x = test_x[:, :, None]
print(test_x.shape)

In [ ]:
###############------------- ORIGINAL ONE WITH 3 LAYERS------------############ RUNNING SUCCESSFully
# def create_model(learning_rate, dense_1, dense_2 ,dense_3):
#     assert learning_rate > 0 and dense_1 > 0 and dense_2 > 0, "Did you set the right configuration?"
#     model = Sequential()
#     model.add(Dense(int(dense_1), input_shape=(13, ), activation='relu', name='fc1'))
#     model.add(Dense(int(dense_2), activation='relu', name='fc2'))
#     model.add(Dense(int(dense_2), activation='relu', name='fc3'))
#     model.add(Dense(1, activation='relu', name='output'))
#     optimizer = SGD(lr=learning_rate)
#     model.compile(optimizer, loss='mae', metrics=['mse'])
#     return model

In [ ]:
###############------------- LSTM ONE WITH 3 LAYERS------------############
def create_model(learning_rate, lstm_1, lstm_2 ,lstm_3):
    model = Sequential()
    model.add(LSTM(int(lstm_1), return_sequences=True, input_shape=(13, 1)))
    model.add(Dropout(0.3))
    model.add(LSTM(int(lstm_2), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(int(lstm_3)))
    model.add(Dropout(0.0))
    model.add(Dense(units=1))
    optimizer = SGD(lr=learning_rate)
    model.compile(optimizer, loss='mae', metrics=['mse'])
    return model

In [ ]:
# model = create_model(learning_rate=0.1, dense_1 = 2, dense_2 = 2,dense_3=2)

In [ ]:
model = create_model(learning_rate=0.1, lstm_1 = 50, lstm_2 = 50,lstm_3=50)

In [ ]:
model.summary()

In [ ]:
#test_model = model.fit(train_x, train_y, validation_data=(test_x, test_y), verbose=1, batch_size = 512, epochs = 20)

In [ ]:
def train_on_iris():
    model = create_model(learning_rate=0.1, lstm_1 = 512, lstm_2 = 512, lstm_3 = 512)
    # This saves the top model. `accuracy` is only available in TF2.0.
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='mse', save_best_only=True, save_freq=2)

    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=1, batch_size=32, epochs=20, callbacks=[checkpoint_callback])
    return model

In [ ]:
#original_model = train_on_iris()

In [ ]:
# Open the file
with open('/content/drive/My Drive/TuneBOHBreport_orig.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    original_model.summary(print_fn=lambda x: fh.write(x + '\n'))

print(original_model.summary())

In [ ]:
import tensorflow.keras as keras
from ray.tune import track

In [ ]:
    # config = {
    #     "num_samples": 10 ,
    #     "config": {
    #         "iterations": 100,
    #     },
    #     "stop": {
    #         "timesteps_total": 100
    #     },
    # }

In [ ]:
class TuneReporterCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        track.log(keras_info=logs, mean_accuracy=logs.get("mse"), mean_loss=logs.get("mae"))


In [ ]:
def tune_iris(config):  # TODO: Change me.
    model = create_model(learning_rate=config["lr"], lstm_1=config["lstm_1"], lstm_2=config["lstm_2"],lstm_3=config["lstm_3"])
    checkpoint_callback = ModelCheckpoint(
        "model.h5", monitor='loss', save_best_only=True, save_freq=2)

    # Enable Tune to make intermediate decisions by using a Tune Callback hook. This is Keras specific.
    callbacks = [checkpoint_callback, TuneReporterCallback()]
    
    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=1, 
        batch_size=32, 
        epochs=20, 
        callbacks=callbacks)

In [ ]:
def tune_iris(config):  # TODO: Change me.
    model = create_model(learning_rate=config["lr"], lstm_1=config["lstm_1"], lstm_2=config["lstm_2"],lstm_3=config["lstm_3"])
    checkpoint_callback = ModelCheckpoint(
        "model.h5", monitor='loss', save_best_only=True, save_freq=2)

    # Enable Tune to make intermediate decisions by using a Tune Callback hook. This is Keras specific.
    callbacks = [checkpoint_callback, TuneReporterCallback()]
    
    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=1, 
        batch_size=512, 
        epochs=10, 
        callbacks=callbacks)

In [ ]:
assert len(inspect.getargspec(tune_iris).args) == 1, "The `tune_iris` function needs to take in the arg `config`."

In [ ]:
print("Test-running to make sure this function will run correctly.")
tune.track.init()  # For testing purposes only.
tune_iris({"lr": 0.1, "lstm_1": 64, "lstm_2": 32, "lstm_3": 16})
print("Success!")

In [ ]:
hyperparameter_space = {
    "lr": tune.loguniform(0.001, 0.1),  
    "lstm_1": tune.uniform(2, 128),
    "lstm_2": tune.uniform(2, 128),
    "lstm_3": tune.uniform(2, 128),
}

In [ ]:
pip install hpbandster ConfigSpace


In [ ]:
# This seeds the hyperparameter sampling.
import numpy as np; np.random.seed(5)  
hyperparameter_space = hyperparameter_space  # TODO: Fill me out.
num_samples = 25

####################################################################################################
################ This is just a validation function for tutorial purposes only. ####################
HP_KEYS = ["lr", "lstm_1", "lstm_2", "lstm_3"]
assert all(key in hyperparameter_space for key in HP_KEYS), (
    "The hyperparameter space is not fully designated. It must include all of {}".format(HP_KEYS))
######################################################################################################

ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
ray.init(log_to_driver=False)
# We clean out the logs before running for a clean visualization later.
! rm -rf ~/ray_results/tune_iris

from ray.tune import Trainable, run
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import ConfigSpace as CS

config_space = CS.ConfigurationSpace()
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lr', lower=0.001, upper=20))
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lstm_1', lower=2, upper=512))
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lstm_2', lower=2, upper=512))
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lstm_3', lower=2, upper=512))

algo = TuneBOHB(
    config_space, max_concurrent=25, metric='keras_info/mse', mode='min')
bohb = HyperBandForBOHB(
    time_attr='training_iteration',
    metric='keras_info/mse',
    mode='min')
analysis= tune.run(tune_iris,search_alg=algo,resources_per_trial = {"cpu": 4, "gpu": 1},num_samples = num_samples)
#analysis = tune.run(tune_iris,search_alg=algo)


assert len(analysis.trials) ==25, "Did you set the correct number of samples?"

In [ ]:
# Obtain the directory where the best model is saved.
print("You can use any of the following columns to get the best model: \n{}.".format(
    [k for k in analysis.dataframe() if k.startswith("keras_info")]))
print("=" * 10)
logdir = analysis.get_best_logdir("keras_info/val_loss", mode="min")
# We saved the model as `model.h5` in the logdir of the trial.
from tensorflow.keras.models import load_model
tuned_model = load_model(logdir + "/model.h5")

In [2]:
# Open the file
with open('/content/drive/My Drive/TuneBOHBreport_tune.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    tuned_model.summary(print_fn=lambda x: fh.write(x + '\n'))

print(tuned_model.summary())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/TuneBOHBreport_tune.txt'

In [ ]:
predicted_value = original_model.predict(test_x)
predicted_value.shape

In [ ]:
test_y.shape

In [ ]:
predicted_value_tuned = tuned_model.predict(test_x)
predicted_value_tuned.shape

In [ ]:
# print("Loss is {:0.4f}".format(tuned_loss))
# print("Tuned Error is {:0.4f}".format(tuned_error))
# #print("The original un-tuned model had an accuracy of {:0.4f}".format(original_accuracy))
# predicted_value = tuned_model.predict(test_x)

In [ ]:
test_y_display=test_y.to_numpy()
print(test_y_display.shape)
print(predicted_value.shape)
print(predicted_value_tuned.shape)
 

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(test_y)
plt.plot(predicted_value)
plt.plot(predicted_value_tuned)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(predicted_value)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(predicted_value_tuned)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test_y)
plt.plot(predicted_value_tuned)
plt.plot(predicted_value)
plt.show()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ~/ray_results/tune_iris